# CMIP6 Precip Analysis (3-hourly data)

In [ ]:
import intake
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6

In [ ]:
cat_url = 'https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/master.yaml'
master_cat = intake.Catalog(cat_url)
list(master_cat)

In [ ]:
cmip_cat = master_cat.cmip6.get()
list(cmip_cat)

In [ ]:
ds = cmip_cat['GISS-E2-1-G.historical.r1i1p1f1.pr'].to_dask()
ds

## Diurnal Cycle Analysis

In [ ]:
def diurnal_cycle(ds):
    return ds.groupby('time.hour').mean(dim='time')

ds_diurnal = ds.groupby('time.year').apply(diurnal_cycle)
ds_diurnal

In [ ]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=20)
cluster

In [ ]:
client = Client(cluster)
ds_diurnal = ds_diurnal.persist(retries=4)

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as crs
from holoviews.operation.datashader import regrid
hv.extension('bokeh', 'matplotlib')

def quick_plot(da, dims, redim_range=None, **user_options):
    options = dict(cmap='viridis', colorbar=True,
                   width=700, height=450)
    options.update(user_options)
    name = da.name
    dataset = hv.Dataset(da)
    image = (dataset.to(hv.QuadMesh, dims, dynamic=True)
                       .options(**options))
    if redim_range is not None:
        image = image.redim.range(**{name: redim_range})

    return hv.output(image, backend='bokeh')

def quick_map(da, dims=['lon', 'lat'], redim_range=None, **user_options):
    options = dict(cmap='viridis', colorbar=True,
                   fig_size=300,
                   projection=crs.Robinson(central_longitude=180))
    options.update(user_options)
    name = da.name
    dataset = gv.Dataset(da)
    image = (dataset.to(gv.Image, dims, dynamic=True)
                       .options(**options))
    if redim_range is not None:
        image = image.redim.range(**{name: redim_range})

    return gv.output(image * gf.coastline(), backend='matplotlib')

### Amplitude of Diurnal Cycle

In [ ]:
pr_diurnal_amplitude = (ds_diurnal.pr.max(dim='hour') -
                        ds_diurnal.pr.min(dim='hour'))

quick_map(pr_diurnal_amplitude, redim_range=(0, 1e-4))

In [ ]:
pr_diurnal_amplitude.mean(dim='lon').transpose().plot(cbar_kwargs={'shrink': 0.5})
plt.title('Amplitude of Diurnal Cycle');

### Phase of Diurnal Cycle

Colors show the hour of maximum precipitation.

_TODO:_ use a proper 2D color wheel to visualize phase an amplitude in one figure.

In [ ]:
pr_diurnal_phase = 3*ds_diurnal.pr.argmax(dim='hour')
quick_map(pr_diurnal_phase, redim_range=(0, 24), cmap='twilight',
           title='Hour of Maximum Diurnal Precip')

In [ ]:
ds.pr.data.size

## Precipitation Intensity Histogram

In [ ]:
def xr_histogram(data, bins, dims, **kwargs):
    
    bins_c = 0.5 * (bins[1:] + bins[:-1]) 
    func = lambda x: np.histogram(x, bins=bins, **kwargs)[0] / x.size

    output_dim_name = data.name + '_bin'
    res = xr.apply_ufunc(func, data,
                         input_core_dims=[dims],
                         output_core_dims=[[output_dim_name]],
                         output_dtypes=['f8'],
                         output_sizes={output_dim_name: len(bins_c)},
                         vectorize=True, dask='parallelized')
    res[output_dim_name] = output_dim_name, bins_c
    res[output_dim_name].attrs.update(data.attrs)
    return res

In [ ]:
bins = np.logspace(-8, -3, 100) 
def func(da):
    da = da.chunk({'lat': 1, 'lon': None, 'time': None})
    return xr_histogram(da, bins, ['lon', 'time'], density=False)
pr_3hr_hist = ds.pr.groupby('time.year').apply(func)
pr_3hr_hist

In [ ]:
pr_3hr_hist.load();

In [ ]:
quick_plot(pr_3hr_hist, ['pr_bin', 'lat'], cmap='OrRd',
           logx=True, redim_range=(0, 0.04), tools=['hover'])

In [ ]:
# this is a bit slow, ~1 minute
# it's creating hundreds of thousands of dask tasks
pr_daily = ds.pr.resample(time='1D').mean(dim='time')
pr_daily

In [ ]:
# this is even slower, takes ~3-4 minutes to show up on the scheduler
# all the tasks have to be processed by the scheduler
pr_daily_hist = pr_daily.groupby('time.year').apply(func)

In [ ]:
pr_daily_hist.load();

In [ ]:
quick_plot(pr_daily_hist, ['pr_bin', 'lat'], cmap='OrRd',
           logx=True, redim_range=(0, 0.04), tools=['hover'])

### Comparison with NCAR GPCP

https://climatedataguide.ucar.edu/climate-data/gpcp-daily-global-precipitation-climatology-project

![ncar](https://climatedataguide.ucar.edu/sites/default/files/styles/node_key_figures_display/public/key_figures/climate_data_set/cdgbutterflygpcp.jpg?itok=RnJ_w-ge)

### Direct Comparison to CPC

_Work in Progress_

This is not the right dataset. But GPCP daily has no opendap endpoint: https://data.nodc.noaa.gov/cgi-bin/iso?id=gov.noaa.ncdc:C00999. Just netCDF files over FTP.

In [ ]:
urls = [f'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/cpc_global_precip/precip.{year}.nc'
        for year in range(1979, 2019)]
cpc = xr.open_mfdataset(urls, parallel=True, chunks={'time': 'auto'})
cpc

In [ ]:
cpc.nbytes/1e9

In [ ]:
cpc = cpc.persist(retries=5)

In [ ]:
cpc.precip[0].plot()

In [ ]:
mm_bins = np.logspace(-3, 4, 100) 
def func_mm(da):
    da = da.chunk({'lat': 1, 'lon': None, 'time': None})
    return xr_histogram(da, mm_bins, ['lon', 'time'], density=False)
cpc_daily_hist = cpc.precip.groupby('time.year').apply(func_mm)
cpc_daily_hist

In [ ]:
cpc_daily_hist.load();

In [ ]:
quick_plot(cpc_daily_hist, ['precip_bin', 'lat'], cmap='OrRd',
           logx=True, tools=['hover'])